In [1]:
import warnings
warnings.filterwarnings("ignore")
from qdrant_client import QdrantClient
import ollama

from config import *
from common import *

In [2]:
# Initialize Ollama client
oclient = ollama.Client(host=Config.HOSTNAME)
# Initialize Qdrant client
qclient = QdrantClient(host=Config.HOSTNAME, port=Config.QDRANT_PORT)

In [3]:
import json
import requests
from typing import Optional
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM

class OllamaLLM(LLM):
    """
    A custom LLM integration for Ollama's API.
    """

    model: str = "tinyllama"  # Default model
    base_url: str = "http://localhost:11434"  # Default Ollama endpoint

    def _call(self, prompt: str, stop: Optional[list] = None) -> str:
        """
        Handles the interaction with the Ollama API.
        """
        payload = {"model": self.model, "prompt": prompt}
        try:
            response = requests.post(
                f"{self.base_url}/api/generate",
                json=payload,
                stream=True,  # Enable streaming
            )
            response.raise_for_status()

            # Process streamed responses
            full_response = ""
            for line in response.iter_lines(decode_unicode=True):
                if line:  # Skip empty lines
                    try:
                        data = json.loads(line)
                        #print("Streaming JSON Object:", data)  # Debugging
                        #print(data)
                        full_response += data.get("response", "")
                        if data.get("done", False):  # Stop when done
                            break
                    except json.JSONDecodeError as e:
                        print(f"Failed to decode line: {line}. Error: {e}")
                        continue

            return full_response
        except requests.RequestException as e:
            raise ValueError(f"Error communicating with Ollama API: {e}")
        except ValueError as e:
            raise ValueError(f"Error processing the response: {e}")

    @property
    def _identifying_params(self) -> dict:
        """Returns identifying parameters for serialization."""
        return {"model": self.model, "base_url": self.base_url}

    @property
    def _llm_type(self) -> str:
        """Type of the LLM."""
        return "ollama"

# Instantiate the Ollama LLM
ollama_llm = OllamaLLM(model="tinyllama", base_url="http://localhost:11434")

In [5]:
ask_query = " Can you tell me more on shared autonomy AI paradigms for human-robot teaming?"
model = "tinyllama"
query_embed , _ = get_embeddings(model, [ask_query], llm_client=oclient)
query_embed = query_embed[0]

hits = retrieve_context(query_embed=query_embed, vdb_client=qclient)
context = ""
for hit in hits:
    #print(hit.payload, "score:", hit.score)
    context += hit.payload['text']


print("Context given to tinyllama >>>>>>")
context = context.replace("\n", " ")
print(context)


# Create a PromptTemplate
topic = "autonomy aii paradigms for human robot teaming"
#query = quert_text

prompt = """
You are a helpful assistant. You will be given a context and a question. Please answer the question based on the context provided. Ensure your answer is clear, concise, and relevant to the information in the context.

Context:
{context}

Topic:
{topic}

Question:
{query}

Answer:
"""

prompt = """
You are a helpful assistant. Could you please answer a question {query} based on the context {context}
"""
prompt_template = PromptTemplate(
    input_variables=["query", "context"],  # Variables to inject
    template=prompt,
)

# chain prompt and llm
chain = prompt_template | ollama_llm
# Run the chain with a specific input
result = chain.invoke({'query': ask_query, 'context': context})
print("LangChain Response:", result)

>>>>>>> localhost 11434
>>>>>>> localhost 6333
Context given to tinyllama >>>>>>
ic systems lies in enabling intuitive control and seamless human-robot interaction when the user's input capabilities are severely limited. many end-users, such as those with impaired motor function or residual muscular control, may have access to only low degrees of freedom (dof) or low-bandwidth control interfaces. however, sought-after robotic assistance tasks such as dexterous object manipulation often requires high-dimensional control in a three-dimensional space. mapping these low-dof user inputs, such as facial gestures, breath control, or limited extremity movements, onto the full 6-dof control space for robotic manipulation presents a highly non-trivial mapping challenge. current high-dof robotic manipulators generally lack the intelligence and flexibility to adaptively address this assistance level tradeoff through appropriate boosting algorithms that can satisfy users with limited motor function

In [ ]:

new_query = "Who is the author of the paper Utility of Accounting for Human Beliefs about AI Intention in Human-AI Collaboration"
topic = "Accounting for Human Beliefs"

new_query += topic


new_embed, _ = get_embeddings(model, [new_query], llm_client=oclient)
new_embed = new_embed[0]

hits = retrieve_context(new_embed, vdb_client=qclient)
new_context = ""
for hit in hits:
    new_context += hit.payload['text']

print(new_context)

result = chain.invoke({'context': new_context, 'topic': topic, 'query': new_query})
print(result)

In [ ]:
# Call the function and print the result
#Config.COLLECTION_NAME = "arvix_papers"
#collections = qclient.get_collections()
#existing_coll = [collection.name for collection in collections.collections]
#print(existing_coll)
#vector_count = get_vector_count(Config.COLLECTION_NAME, vdb_client=qclient)
#print(f"Number of vectors in the collection '{Config.COLLECTION_NAME}': {vector_count}")